# Model Fitting: ESS (Elliptical Slice Sampling)

In [4]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set_style("white")
import time
import timeit

import scipy.stats
# import scipy.stats.norm as norm
import pandas as pd
import pymc as pm

import re
import numpy as np
import pickle as pickle

In [5]:
player_info = pd.read_csv("clean-data/player_info_pergame.csv")
game_outcomes = pd.read_csv("clean-data/game_outcomes_15-16.csv")

In [6]:
N_players = len(player_info)
N_teams = np.max(game_outcomes['Visitor_Index'].values) + 1
N_games =  len(game_outcomes)

print N_players, N_teams, N_games

476 30 1230


In [7]:
# # print np.max(game_outcomes['Visitor_Index'].values)
# # print np.max(game_outcomes['Home_Index'].values)

# a = np.array([[1,2,3,4], [1,2,3,4]])
# print a.shape
# print a.mean(1)
# print a[0][0:]

# b = np.zeros(a.shape)
# print b
print np.arange(10)

[0 1 2 3 4 5 6 7 8 9]


In [7]:
def correlation()
    dim = 2*(N_teams + N_players)
    mat = np.identity(dim)
    
    # i: team index
    for i in np.arange(N_teams):
        
        
    


    
print np.random.gamma(1.0, scale=1.0, size=None)



#elliptical slice sampler (Murray, Adams, and MacKay JMLR 2010)
# Inputs:
# Y: observed data
# Sig: covariance matrix of the prior
# log_lik: the log likelihood function
# N_mcmc: number of desired samples
# burn_in: number of burn in iterations
# N: dimensionality of the input, this should be M+1 because alpha is part of Sigma

# returns an array of samples of size N_mcmc x N

    
def ess(Y, Sig,log_lik, N_mcmc, burn_in, N):
    print 'running elliptical slice sampling...'
    mcmc_samples = np.zeros((N_mcmc+burn_in, N))
    norm_samples = np.random.multivariate_normal(np.zeros(N), Sig, N_mcmc+burn_in)
    unif_samples = np.random.uniform(0,1,N_mcmc+burn_in)
    theta = np.random.uniform(0,2*np.pi,N_mcmc+burn_in)
    theta_min = theta-2*np.pi
    theta_max = theta+2*np.pi
    for i in range(1,N_mcmc+burn_in):
        f = mcmc_samples[i-1,:]
        llh_thresh = log_lik(f,Y) + np.log(unif_samples[i])
        f_star = f*np.cos(theta[i])+norm_samples[i,:]*np.sin(theta[i])
        while log_lik(f_star,Y) < llh_thresh:
            if theta[i] < 0:
                theta_min[i] = theta[i]
            else:
                theta_max[i] = theta[i]
            theta[i] = np.random.uniform(theta_min[i],theta_max[i],1)  A
            f_star = f*np.cos(theta[i])+norm_samples[i,:]*np.sin(theta[i]) 		
        mcmc_samples[i,:] = f_star
    return mcmc_samples[(burn_in+1):(burn_in+N_mcmc),]

0.903188987514


In [19]:
################# INPUT SECTION ###################
###################################################

########### player indicator for each game ###############
### host_lineup_arr & guest_lineup_arr are (1230 x 476)###
with open("clean-data/host_team_line_up.pkl", "rb") as f:
    host_team_line_up = pickle.load(f)

with open("clean-data/guest_team_line_up.pkl", "rb") as f:
    guest_team_line_up = pickle.load(f)
    
# print len(host_team_line_up[0])
host_lineup_arr = np.array(host_team_line_up)
guest_lineup_arr = np.array(guest_team_line_up)


######### team indicator for each game ########### 
### host_matrix & guest_matrix are (1230 x 30) ###
guest_matrix = np.zeros((game_outcomes.shape[0], np.max(game_outcomes['Visitor_Index']) + 1), dtype = bool)
guest_matrix.shape
host_matrix = np.copy(guest_matrix)

def make_matrix(mat, indices):
    for (i, ind) in enumerate(indices):
        mat[i, ind] = True

make_matrix(host_matrix, game_outcomes['Visitor_Index'].values)
make_matrix(guest_matrix, game_outcomes['Home_Index'].values)


############## Observed data ##################
score_diff = game_outcomes['diff'].values
off_rating = player_info['PTS'].values + player_info['AST'].values
def_rating = player_info['BLK'].values + player_info["STL"].values + player_info['DRB'].values

In [20]:
def split_params(coefs, nplayers, nteams):
    '''
    Split the parameters
    first are the beta0 for each team
    then the beta for each player
    then the gamma0 for each team
    then the gamma for each player'''
    assert(coefs.shape == (2*(nplayers+nteams),))
    
    beta0 = coefs[:nteams]
    beta_player = coefs[nteams:(nplayers + nteams)]
    gamma0 = coefs[(nplayers + nteams):(nplayers + 2*nteams)]
    gamma_player = coefs[(nplayers + 2*nteams):]
    
    # parameterize sigma by its log
#     logsigma = coefs[-1]
    
    assert(beta0.shape == (nteams,))
    assert(beta_player.shape == (nplayers,))
    assert(beta0.shape == (nteams,))
    assert(gamma_player.shape == (nplayers,))
    
    return (beta0, beta_player, gamma0, gamma_player)

In [21]:

def diff_score_calc(coefs):
    # This function returns the differential score for each game given the states of the latent vairables
    
    beta0, betas, gamma0, gammas = \
        split_params(coefs, N_players, N_teams)
        
    guest_off_0 = np.dot(guest_matrix, beta0)
    guest_def_0 = np.dot(guest_matrix, gamma0)
    host_off_0 = np.dot(host_matrix, beta0)
    host_def_0 = np.dot(host_matrix, gamma0)
    
    guest_off = guest_off_0 + np.dot(guest_lineup_arr, betas * off_rating)
    guest_def = guest_def_0 + np.dot(guest_lineup_arr, gammas * def_rating)
    host_off = host_off_0 + np.dot(host_lineup_arr, betas * off_rating)
    host_def = host_def_0 + np.dot(host_lineup_arr, gammas * def_rating)
    
    mean = guest_off - host_def - (host_off - guest_def)
    return mean

In [22]:
def logLik(coefs, lam = 0, sigma = 10.):
    # This functino calculates the logLikelihood for the entire games
    mean = diff_score_calc(coefs)
    loglik = np.sum(scipy.stats.norm.logpdf(score_diff, mean, sigma))
    
    # add L2 regularization
    loglik -= lam * np.dot(coefs, coefs)
    return (loglik)

In [23]:
def changeCoefs(coefs, theta, norm_random):
    ''' evaluate the negative log likelihood for our basketball model '''
    
    beta0, betas, gamma0, gammas = \
            split_params(coefs, N_players, N_teams)    
    
    norm_beta0, norm_betas, norm_gamma0, norm_gammas = \
            split_params(norm_random, N_players, N_teams)
    
#     nteams = len(beta0)
#     nplayers = len(betas)
    
    new_coefs = np.zeros(coefs.shape)    
    new_coefs[:nteams] = beta0 * np.cos(theta) + norm_beta0 * np.sin(theta)
    new_coefs[nteams:(nplayers + nteams)] = betas * np.cos(theta) + norm_betas * np.sin(theta)
    new_coefs[(nplayers + nteams):(nplayers + 2*nteams)] = gamma0 * np.cos(theta) + norm_gamma0 * np.sin(theta)
    new_coefs[(nplayers + 2*nteams):] = gammas * np.cos(theta) + norm_gammas * np.sin(theta)
    
    return new_coefs

In [24]:
nplayers = player_info.shape[0]
nteams = host_matrix.shape[1]
# nteams = N_teams

# coefs = np.random.randn(2*(nplayers + nteams))
# b0, bs, g0, gs = split_params(coefs, nplayers, nteams)

# host_lineup_arr = np.array(host_team_line_up)
# guest_lineup_arr = np.array(guest_team_line_up)

In [25]:
# a = np.random.rand(5)
# temp = np.zeros((5,5))
# temp[0,:] = a
# print a
# print temp

# mvn = np.random.multivariate_normal

# print mvn(np.zeros(3), np.identity(3), 5)

# a = np.array([1,2,3,4])
# b = np.array([1,2,3,4])
# print a/b

In [28]:
mvn = np.random.multivariate_normal
def ess(logLik, N_mcmc, burn_in):
    bi = burn_in

    # Make a random initial proposal for the states of the latent variables
    # based on the assumption that the states are in the Gaussian distribution
    mcmc_coefs = np.random.randn(N_mcmc + bi, 2*(N_players + N_teams))
          
    dim = 2*(N_teams + N_players)
    norm_random = mvn(np.zeros(dim), np.identity(dim), N_mcmc+bi)
    
    # random draw from unifrom distribution with which we'll determine 
    # the loglikelihood threshold (likelihood threshold defines the 'slice' where we sample)
    unif_samples = np.random.uniform(0, 1, N_mcmc+bi)
    
    # initial proposal of the theta
    theta = np.random.uniform(0, 2*np.pi, N_mcmc+bi)
    
    # variables with which we'll propose a new state by shrinking the range of theta
    theta_min = theta - 2*np.pi
    theta_max = theta + 2*np.pi
    
    # We select a new location (i.e. new state of the latent variables)
    # on the randomly generated ellipse given theta and norm_samples
    for i in range(1, N_mcmc + bi):
#         if i % 100 == 0:
#             print i
     
        coefs = mcmc_coefs[i - 1, :]

        # the loglikelihood threshold
        # the threshold is chosen between [0, Likelihood]
        llh_thresh = logLik(coefs) + np.log(unif_samples[i])
        
        # new proposals for the latent variable states
        new_coefs = changeCoefs(coefs, theta[i], norm_random[i,:])
        while logLik(new_coefs) < llh_thresh:
            if theta[i] < 0:
                theta_min[i] = theta[i]
            else:
                theta_max[i] = theta[i]

            # shrink the range of the search for the new proposal while we logLikelohood > threshold
            theta[i] = np.random.uniform(theta_min[i], theta_max[i], 1)  
            
            # new proposals with the changed range of the ellipse
            new_coefs = coefs*np.cos(theta[i]) + norm_random[i,:]*np.sin(theta[i])
        
        # when the condition is satisfied, we keep the proposals of the latent variable states
        mcmc_coefs[i, :] = new_coefs

    # return except the burnin     
    return mcmc_coefs[(bi + 1): (bi + N_mcmc),]

# Estimation of the latent variable states

In [29]:
# game_index = 10
N_mcmc = 2000
burn_in = 100

start_time = time.time()

stateCoefs = ess(logLik, N_mcmc, burn_in)

elapsed = time.time() - start_time
print("Elapsed Time (sec): %f" %elapsed)

mean_coefs = stateCoefs.mean(axis=0)
mean_b0, mean_bs, mean_g0, mean_gs = split_params(mean_coefs, N_players, N_teams)

print mean_b0.shape 
print mean_bs.shape
print mean_g0.shape
print mean_gs.shape

Elapsed Time (sec): 159.581000
(30L,)
(476L,)
(30L,)
(476L,)


In [16]:
player_info['Tm'].values[0]

'Los Angeles Lakers'

In [31]:
contribute_off = np.argsort(-mean_bs)# * player_info['PTS'])

betas_df = player_info.loc[contribute_off]
betas_df['beta'] = mean_bs[contribute_off]
betas_df.head(20)

,Player,Tm,2015-16,Pos,Age,G,GS,MP,FG,FGA,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,beta
133,Josh McRoberts,Miami Heat,5543725,PF,28,42,1,14.2,1.3,3.5,...,0.5,2.0,2.5,1.9,0.4,0.2,0.7,1.6,3.6,3.369782
403,Joe Harris,Orlando Magic,845059,SG,24,5,0,3.0,0.2,0.8,...,0.0,0.6,0.6,0.4,0.0,0.0,0.2,0.2,0.6,2.584807
374,Rakeem Christmas,Indiana Pacers,1000000,PF,24,1,0,6.0,2.0,2.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,4.0,2.563499
437,Lamar Patterson,Atlanta Hawks,525093,SG,24,35,0,11.3,0.8,2.3,...,0.1,1.3,1.4,1.1,0.3,0.1,0.7,1.3,2.4,2.201607
94,O.J. Mayo,Milwaukee Bucks,8000000,SG,28,41,24,26.6,2.9,7.8,...,0.3,2.2,2.6,2.9,1.2,0.2,1.8,3.1,7.8,2.099528
342,P.J. Hairston,Memphis Grizzlies,1201440,SF,23,66,52,19.9,2.2,6.2,...,0.4,2.3,2.6,0.6,0.5,0.2,0.6,2.1,6.2,2.056974
95,George Hill,Indiana Pacers,8000000,PG,29,74,73,34.1,4.4,10.0,...,0.8,3.2,4.0,3.5,1.1,0.2,1.4,2.0,12.1,2.029025
118,Jason Thompson,Golden State Warriors,6431250,C,29,47,7,10.0,1.1,2.3,...,0.7,2.2,2.9,0.6,0.3,0.4,0.3,1.7,3.1,1.991593
155,Chase Budinger,Indiana Pacers,5000000,SF,27,66,2,14.1,1.6,3.6,...,0.7,1.6,2.3,1.0,0.5,0.2,0.7,0.9,4.1,1.988822
179,Alan Anderson,Washington Wizards,4000000,SG,33,13,0,14.8,1.6,4.5,...,0.1,2.0,2.1,1.1,0.3,0.1,0.2,1.9,5.0,1.968427


In [16]:
betas_df[betas_df['Tm'] == "New Orleans Pelicans"]

,Player,Tm,2015-16,Pos,Age,G,GS,MP,FG,FGA,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,beta
170,Alexis Ajinca,New Orleans Pelicans,4300000,C,27,59,17,14.6,2.5,5.3,...,1.3,3.3,4.6,0.5,0.3,0.6,0.9,2.3,6.0,2.549573
473,Nate Robinson,New Orleans Pelicans,44000,PG,31,2,1,11.5,0.0,0.5,...,0.0,0.0,0.0,2.0,0.5,0.0,0.0,2.5,0.0,1.538746
67,Tyreke Evans,New Orleans Pelicans,11227000,SG,26,25,25,30.6,5.4,12.6,...,0.8,4.4,5.2,6.6,1.3,0.3,2.9,2.6,15.2,1.532877
348,Toney Douglas,New Orleans Pelicans,1164858,PG,29,61,18,20.7,3.0,7.3,...,0.4,1.9,2.3,2.6,1.1,0.1,1.0,2.1,8.7,1.072515
304,Kendrick Perkins,New Orleans Pelicans,1499200,C,31,37,5,14.6,1.1,2.0,...,0.7,2.8,3.5,0.8,0.3,0.3,1.1,1.7,2.5,1.031074
71,Jrue Holiday,New Orleans Pelicans,10595507,PG,25,65,23,28.2,6.3,14.4,...,0.4,2.6,3.0,6.0,1.4,0.3,2.6,2.3,16.8,1.024446
222,Dante Cunningham,New Orleans Pelicans,3000000,SF,28,80,46,24.6,2.3,5.2,...,0.7,2.2,3.0,1.0,0.5,0.4,0.5,2.2,6.1,0.876138
216,Norris Cole,New Orleans Pelicans,3036927,PG,27,45,23,26.6,4.4,10.8,...,0.2,3.1,3.4,3.7,0.8,0.1,1.7,2.3,10.6,0.557321
365,Luke Babbitt,New Orleans Pelicans,1100000,SF,26,47,13,18.0,2.6,6.1,...,0.5,2.6,3.1,1.1,0.2,0.1,0.5,1.9,7.0,-0.104882
105,Anthony Davis,New Orleans Pelicans,7070730,PF,22,61,61,35.5,9.2,18.6,...,2.1,8.1,10.3,1.9,1.3,2.0,2.0,2.4,24.3,-0.416855


In [17]:
# diff. score given estiamted coefficients
estimated = diff_score_calc(mean_coefs)

# difference between the estimatino and observed data
diff = score_diff - estimated

print estimated
print score_diff
print diff.shape
print diff
print 100.*diff/score_diff

# print 'the actual differential score = %s' %(real)
# print 'the differential score with the estimated variable states = %s' %(estimated)
# print 'difference is %s percent' %(np.fabs(real - estimated)*100.0/real)

[ -3.73434872   6.27702626 -14.58508205 ...,  10.84106253  -3.87634704
   4.53095297]
[ 12.  -2. -16. ...,  -9.  -8. -11.]
(1230L,)
[ 15.73434872  -8.27702626  -1.41491795 ..., -19.84106253  -4.12365296
 -15.53095297]
[ 131.11957269  413.85131308    8.84323716 ...,  220.45625037   51.54566198
  141.19048154]
